In [37]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain.vectorstores.chroma import Chroma


In [3]:
DATA_PATH = "./data"

In [4]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [6]:
documents = load_documents()

In [28]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False
    )
    return text_splitter.split_documents(documents)

In [29]:
documents = load_documents()
chunks = split_documents(documents)

In [43]:
def get_embedding_function():
    embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name='us-east-1'
    )
    return embeddings

In [42]:
chunks[0].metadata.get('source')

'data\\Методика ВНИИГАЗ.pdf'

In [44]:
from langchain_aws.embeddings.bedrock import BedrockEmbeddings

In [45]:
embeddings = BedrockEmbeddings(credentials_profile_name="bedrock-admin", region_name="us-east-1")

ValidationError: 1 validation error for BedrockEmbeddings
  Value error, Could not load credentials to authenticate with AWS client. Please check that credentials in the specified profile name are valid. Bedrock error: The config profile (bedrock-admin) could not be found [type=value_error, input_value={'credentials_profile_nam...gion_name': 'us-east-1'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [8]:
import os
import getpass

In [9]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

In [1]:
urls = [
    "https://education.yandex.ru/handbook/ml/article/vvedenie-v-generativnoe-modelirovanie",
    "https://education.yandex.ru/handbook/ml/article/variational-autoencoder-(vae)",
    "https://education.yandex.ru/handbook/ml/article/generativno-sostyazatelnye-seti-(gan)"
]

In [3]:
# load docs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [4]:
# load docs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [5]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200)

In [6]:
doc_splits = text_splitter.split_documents(docs_list)

In [14]:
_set_env("NOMIC_API_KEY")

In [15]:
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model='nomic-embed-text-v1.5'))

In [16]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

In [20]:
retriever.invoke('Диффузионные модели')

[Document(metadata={'id': 'df90f4f1-beff-41eb-94da-a01dd829ca8b', 'source': 'https://education.yandex.ru/handbook/ml/article/variational-autoencoder-(vae)', 'title': 'Variational Autoencoder (VAE)', 'description': 'Variational Autoencoder (VAE)', 'language': 'ru'}, page_content='Инференс\nНа этапе инференса в модель подаются токены текстового описания картинки, и на их основании модель авторегрессионно предсказывает кодовые векторы:\n\nКодовые векторы картинки подаются в декодер dVAE, который отображает их в финальную картинку:'),
 Document(metadata={'id': 'aeb58c69-7daa-4d6d-9483-0b3d87d8f0f0', 'source': 'https://education.yandex.ru/handbook/ml/article/vvedenie-v-generativnoe-modelirovanie', 'title': 'Введение в генеративное моделирование', 'description': 'Введение в генеративное моделирование - Хендбук от Яндекс.Образования. Откройте для себя передовые подходы, практические советы и вдохновляющие идеи от наших экспертов.', 'language': 'ru'}, page_content='Явное моделирование. В этом 